# Data Pre-processing

In [ ]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning) # ignore warning messages
warnings.filterwarnings(action='ignore', category=UserWarning)

In [ ]:
air = pd.read_csv('Edata.csv')

C:\Users\debar\AppData\Local\Temp\ipykernel_9960\2827644914.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  air = pd.read_csv('E:/PrepInsta Winter Internship Program/Week 7/data-1.csv')


In [ ]:
# retrieving the unique values in the 'type' column of the "air" dataframe
air['type'].unique()

array(['Residential, Rural and other Areas', 'Industrial Area', nan,
       'Sensitive Area', 'Industrial Areas', 'Residential and others',
       'Sensitive Areas', 'Industrial', 'Residential', 'RIRUO',
       'Sensitive'], dtype=object)

In [ ]:
# replacing multiple values in the 'type' column of the dataframe
air['type'].replace('Residential, Rural and other Areas','Residential',inplace = True)
air['type'].replace('Residential and others','Residential',inplace = True)
air['type'].replace('Industrial Areas','Industrial',inplace = True)
air['type'].replace('Industrial Area','Industrial',inplace = True)
air['type'].replace('Sensitive Area','Sensitive',inplace = True)
air['type'].replace('Sensitive Areas','Sensitive',inplace = True)

In [ ]:
# retrieving the unique values in the 'state' column of the 'air' DataFrame
air['state'].unique()

array(['Andhra Pradesh', 'Arunachal Pradesh', 'Assam', 'Bihar',
       'Chandigarh', 'Chhattisgarh', 'Dadra & Nagar Haveli',
       'Daman & Diu', 'Delhi', 'Goa', 'Gujarat', 'Haryana',
       'Himachal Pradesh', 'Jammu & Kashmir', 'Jharkhand', 'Karnataka',
       'Kerala', 'Madhya Pradesh', 'Maharashtra', 'Manipur', 'Meghalaya',
       'Mizoram', 'Nagaland', 'Odisha', 'Puducherry', 'Punjab',
       'Rajasthan', 'Sikkim', 'Tamil Nadu', 'Telangana', 'Uttar Pradesh',
       'Uttarakhand', 'Uttaranchal', 'West Bengal',
       'andaman-and-nicobar-islands', 'Lakshadweep', 'Tripura'],
      dtype=object)

In [ ]:
air['state'].replace('andaman-and-nicobar-islands', 'Andaman and Nicobar Islands', inplace=True)

In [ ]:
# Converting the 'date' column to datetime format and extract the 'year' column
air['date'] = pd.to_datetime(air['date'],format='mixed')
air['year'] = air['date'].dt.year

In [ ]:
# Filling the missing 'year' values using ffill and converting them to integer type
air['year'].fillna(method='ffill', inplace=True)
air['year'] = air['year'].astype(int)

In [ ]:
# checking null values in year column
print("Null Values in Year column:", air['year'].isnull().sum())

Null Values in Year column: 0


In [ ]:
# Creating a separate dataFrame to show the proportion of missing values of each column
missing = pd.DataFrame(air.isna().sum() / len(air))
missing.columns = ['Proportion']

In [ ]:
# Displaying the columns sorted by the proportion of missing values in descending order
print(missing.sort_values(by='Proportion', ascending=False).round(3))

                             Proportion
pm2_5                             0.979
spm                               0.545
agency                            0.343
stn_code                          0.331
rspm                              0.092
so2                               0.080
location_monitoring_station       0.063
no2                               0.037
type                              0.012
date                              0.000
sampling_date                     0.000
location                          0.000
state                             0.000
year                              0.000


In [ ]:
pollutants = ['pm2_5', 'no2', 'so2', 'spm', 'rspm']

def get_state_median(state):
    # calculate the median values for no2 for the given state, grouped by type
    state_medians = air[air['state'] == state].groupby('type')[pollutants].median().reset_index()

    # return the state_medians dataframe
    return state_medians

In [ ]:
get_state_median('Andhra Pradesh')

,type,pm2_5,no2,so2,spm,rspm
0,Industrial,NaN,22.2,5.4,214.0,76.0
1,Residential,NaN,20.0,5.0,192.0,78.0
2,Sensitive,NaN,13.0,4.6,130.0,51.0


In [ ]:
median_values = get_state_median('Andhra Pradesh')

In [ ]:
# Fill missing 'so2' values in 'Andhra Pradesh' for Industrial, Residential, and Sensitive types
air.loc[(air['state'] == 'Andhra Pradesh') & (air['type'].isin(['Industrial', 'Residential', 'Sensitive'])), 'so2'] = median_values['so2']

In [ ]:
# Fill missing 'no2' values in 'Andhra Pradesh' for Industrial, Residential, and Sensitive types
air.loc[(air['state'] == 'Andhra Pradesh') & (air['type'].isin(['Industrial', 'Residential', 'Sensitive'])), 'no2'] = median_values['so2']

In [ ]:
print("Missing Values in rspm:", air['rspm'].isnull().sum())
print("Missing values in spm:" ,air['spm'].isnull().sum())

Missing Values in rspm: 40222
Missing values in spm: 237387


In [ ]:
df1 = dict(list(air.groupby(['location', 'type'])))
data = pd.DataFrame()

# Iterate through groups, sort by 'date', and forward-fill 'rspm' and 'spm' values
for key in df1:
    df2 = df1[key].sort_values('date')
    df2['rspm'].fillna(method='ffill', inplace=True)
    df2['spm'].fillna(method='ffill', inplace=True)
    data = pd.concat([data, df2])

In [ ]:
# Group 'data' DataFrame by 'location' and 'type' and create a new DataFrame with backward-filled 'rspm' and 'spm' values
df1 = dict(list(data.groupby(['location', 'type'])))
data1 = pd.DataFrame()

# Iterate through groups, sort by 'date', and backward-fill 'rspm' and 'spm' values
for key in df1:
    df2 = df1[key].sort_values('date')
    df2['rspm'].fillna(method='bfill', inplace=True)
    df2['spm'].fillna(method='bfill', inplace=True)
    data1 = pd.concat([data1, df2])

In [ ]:
# Print the number of missing values in the 'rspm' and 'spm' columns of the 'data1' DataFrame
print("Missing values in the 'data1' df")
print("rspm:", data1['rspm'].isnull().sum())
print("spm:", data1['spm'].isnull().sum())

Missing values in the 'data1' df
rspm: 4102
spm: 47909


In [ ]:
# Group 'data1' DataFrame by 'state' and 'type' and create a new DataFrame with median-filled 'rspm' and 'spm' values
df1 = dict(list(data1.groupby(['state', 'type'])))
data2 = pd.DataFrame()

# Iterate through groups and fill missing 'rspm' and 'spm' values with group-wise medians
for key in df1:
    df2 = df1[key]
    df2['rspm'].fillna(df2['rspm'].median(), inplace=True)
    df2['spm'].fillna(df2['spm'].median(), inplace=True)
    data2 = pd.concat([data2, df2])

In [ ]:
# Print the number of missing values in the 'rspm' and 'spm' columns of the 'data2' DataFrame
print("Missing values in the 'data2' df")
print("rspm:", data2['rspm'].isnull().sum())
print("spm:", data2['spm'].isnull().sum())

Missing values in the 'data2' df
rspm: 182
spm: 1972


In [ ]:
# Group 'data2' DataFrame by 'type' and create a new DataFrame with median-filled 'rspm' and 'spm' values
df1 = dict(list(data2.groupby('type')))
data3 = pd.DataFrame()

# Iterate through groups and fill missing 'rspm' and 'spm' values with group-wise medians
for key in df1:
    df2 = df1[key]
    df2['rspm'].fillna(df2['rspm'].median(), inplace=True)
    df2['spm'].fillna(df2['spm'].median(), inplace=True)
    data3 = pd.concat([data3, df2])

In [ ]:
# Print the number of missing values in the 'rspm' and 'spm' columns of the 'data3' DataFrame
print("Missing values in the data3 df")
print("rspm:", data3['rspm'].isnull().sum())
print("spm:", data3['spm'].isnull().sum())

Missing values in the data3 df
rspm: 0
spm: 1304


In [ ]:
# Display the count of each type in the 'data3' DataFrame
data3['type'].value_counts()

type
Residential    265963
Industrial     148071
Sensitive       15011
RIRUO            1304
Name: count, dtype: int64

In [ ]:
# Reset index and drop unnecessary columns from the 'data3' DataFrame
data3.reset_index(inplace=True)
data3.drop(columns=['index', 'stn_code', 'sampling_date', 'agency', 'location_monitoring_station'], inplace=True)
data3.head()

,state,location,type,so2,no2,rspm,spm,pm2_5,date,year
0,Andhra Pradesh,Hyderabad,Industrial,5.0,5.0,90.3,82.0,NaN,1990-01-02,1990
1,Andhra Pradesh,Hyderabad,Industrial,NaN,NaN,90.3,82.0,NaN,1990-01-03,1990
2,Andhra Pradesh,Hyderabad,Industrial,NaN,NaN,90.3,82.0,NaN,1990-01-04,1990
3,Andhra Pradesh,Hyderabad,Industrial,NaN,NaN,90.3,82.0,NaN,1990-01-05,1990
4,Andhra Pradesh,Hyderabad,Industrial,NaN,NaN,90.3,82.0,NaN,1990-01-06,1990


In [ ]:
data3.isnull().sum()

state            0
location         0
type             0
so2          59271
no2          41067
rspm             0
spm           1304
pm2_5       421035
date             4
year             0
dtype: int64

In [ ]:
# Save data3 to csv
data3.to_csv('air_quality_cleaned.csv', index=False)